In [38]:
import requests
import json
import pandas as pd
import boto
from io import StringIO


## Get all vehicles directly from netstar

x_api_key = "NETSTAR API KEY"
url = "https://ubi-ms.netstar.co.za/vehicle/vehicles/"
headers = {'x-api-key' : x_api_key ,'content-type' : 'application/json'}

result_vehicles = requests.post(url, headers=headers)

raw_parsed = result_vehicles.text
json_object = json.loads(raw_parsed)

df1 = pd.json_normalize(json_object)




## Get all the folders in S3 bucket netstar-south-africa-rabbit

aws_id = "INSERT AWS ID"
aws_secret = "INSERT AWS SECRET"

s3conn = boto.connect_s3(aws_id, aws_secret)
bucket_name = 'netstar-south-africa-rabbit'
bucket = s3conn.get_bucket(bucket_name)
folders = bucket.list('', '/')

cars = []
for folder in folders:
    cars.append(folder.name.replace("/",""))

    
    
    
## Check if each car has had a heartbeat    
    
df1["Heartbeat"] = ""
for index, row in df1.iterrows():
    if str(row["VbuNo"]) in cars:
        df1.loc[index,'Heartbeat']="Yes"
df1.to_csv('/Users/rasmuserlemann/Documents/all_netstar_cars.csv')